In [1]:
%load_ext autoreload
%autoreload 2
    
import dotenv
import langchain
from interlab_zoo.resolver.resolver import load_people, resolve, resolve_pairs
from interlab.tracing import FileStorage, TracingNode
import pandas as pd
dotenv.load_dotenv()

True

In [5]:
storage = FileStorage("logs/")
storage.start_server(port=5000)

<ServerHandle http://localhost:5000>

In [2]:
import langchain
from langchain.cache import InMemoryCache
langchain.llm_cache = InMemoryCache()

In [3]:
people = load_people("people.json")

In [ ]:
model = langchain.chat_models.ChatOpenAI(model_name='gpt-3.5-turbo')
result = []
with TracingNode("movie_exp", storage=storage) as root:
    for i in range(12):
        with TracingNode(f"round {i}"):
            result += resolve_pairs(model, people)

root.display()

In [87]:
data = []

for r in result:    
    for k in ["just_answer", "with_history"]:
        for s in r.scores[k]:
            data.append((k, s.me, s.other, s.score, r.steps))

df = pd.DataFrame(data, columns=["kind", "me", "other", "score", "steps"])
df[df.kind == "just_answer"].shape

(288, 5)

In [90]:
ja = df[df.kind == "just_answer"][["me", "other", "score"]].groupby(["me", "other"]).mean().unstack()
wh = df[df.kind == "with_history"][["me", "other", "score"]].groupby(["me", "other"]).mean().unstack()

display(ja)
display(wh)
display(wh - ja)

st = df[["me", "other", "steps"]].groupby(["me", "other"]).mean().unstack()
display(st)

score                  
other  Connor Edward Lily Maria
me                             
Connor    NaN    9.0  8.5   7.0
Edward    8.0    NaN  8.0   8.0
Lily      8.5    8.5  NaN   9.5
Maria     8.5    9.0  9.0   NaN

score                   
other  Connor Edward  Lily Maria
me                              
Connor    NaN    8.5   8.5   8.5
Edward    9.0    NaN   9.0   9.5
Lily      9.0    9.0   NaN  10.0
Maria     9.0   10.0  10.0   NaN

score                  
other  Connor Edward Lily Maria
me                             
Connor    NaN   -0.5  0.0   1.5
Edward    1.0    NaN  1.0   1.5
Lily      0.5    0.5  NaN   0.5
Maria     0.5    1.0  1.0   NaN

steps                  
other  Connor Edward Lily Maria
me                             
Connor    NaN    2.0  2.0   2.0
Edward    2.0    NaN  2.0   2.5
Lily      2.0    2.0  NaN   2.0
Maria     2.0    2.5  2.0   NaN